<a href="https://colab.research.google.com/github/qmeng222/MNIST-Playground/blob/main/measure_performance_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task:
Compute performance time
1.   Exact computation: use system clock, accurate to sub-millisecond.
2.   Rough computation: use timer at the bottom of colab screen, accurate to seconds.

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import time

import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [8]:
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# extract labels, then remove labels from data:
labels = data[:,0]
data   = data[:,1:]

# normalize all data (train & test) to [0 1]:
dataNorm = data / np.max(data)

In [9]:
# Step 1: np array -> torch tensor
dataT   = torch.tensor( dataNorm ).float()
labelsT = torch.tensor( labels ).long()

# Step 2: use scikitlearn to split the data
train_data,test_data, train_labels,test_labels = train_test_split(dataT, labelsT, test_size=.1)

# Step 3: torch tensor -> torch dataset obj
train_data = torch.utils.data.TensorDataset(train_data,train_labels)
test_data  = torch.utils.data.TensorDataset(test_data,test_labels)

# Step 4: torch dataset obj -> torch dataloader obj
batchsize  = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [10]:
def createTheMNISTNet():

  # model class:
  class mnistNet(nn.Module):
    def __init__(self):
      super().__init__()

      ### input layer
      self.input = nn.Linear(784,64)

      ### hidden layer
      self.fc1 = nn.Linear(64,32)
      self.fc2 = nn.Linear(32,32)

      ### output layer
      self.output = nn.Linear(32,10)

    # forward pass
    def forward(self,x):
      x = F.relu( self.input(x) )
      x = F.relu( self.fc1(x) )
      x = F.relu( self.fc2(x) )
      return self.output(x)

  # create the model instance
  net = mnistNet()

  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.01)

  return net,lossfun,optimizer

In [11]:
def function2trainTheModel():

  # start the timer!
  timerInFunction = time.process_time()

  numepochs = 10

  # create a new model
  net,lossfun,optimizer = createTheMNISTNet()

  # initialize losses
  losses    = torch.zeros(numepochs)
  trainAcc  = []
  testAcc   = []

  # loop over epochs
  for epochi in range(numepochs):
    batchAcc  = []
    batchLoss = []

    # loop over batches:
    for X,y in train_loader:
      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = net(X)

    # compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

    # computate complete time at the end of each epoch:
    comptime = time.process_time() - timerInFunction
    print(f'Epoch {epochi+1}/{numepochs}, elapsed time: {comptime:.2f} sec, test accuracy {testAcc[-1]:.0f}%')

  # end epochs

  # function output
  return trainAcc,testAcc,losses,net

In [12]:
trainAcc,testAcc,losses,net = function2trainTheModel()

Epoch 1/10, elapsed time: 1.31 sec, test accuracy 93%
Epoch 2/10, elapsed time: 3.24 sec, test accuracy 91%
Epoch 3/10, elapsed time: 4.63 sec, test accuracy 94%
Epoch 4/10, elapsed time: 6.02 sec, test accuracy 93%
Epoch 5/10, elapsed time: 7.40 sec, test accuracy 94%
Epoch 6/10, elapsed time: 8.75 sec, test accuracy 94%
Epoch 7/10, elapsed time: 10.16 sec, test accuracy 95%
Epoch 8/10, elapsed time: 11.57 sec, test accuracy 94%
Epoch 9/10, elapsed time: 12.94 sec, test accuracy 95%
Epoch 10/10, elapsed time: 14.78 sec, test accuracy 96%


👆 Rough performance time from colab is 15s, roughly matching the elapsed time for the last epoch, as printed above.

In [13]:
# now run a second timer over repeated iterations

# start another timer with a different variable name:
timerOutsideFunction = time.process_time()

for i in range(10):
  function2trainTheModel()

TotalExperimentTime = time.process_time() - timerOutsideFunction
print(f'\n\n\nTotal elapsed experiment time: {TotalExperimentTime/60:.2f} minutes')

Epoch 1/10, elapsed time: 1.65 sec, test accuracy 92%
Epoch 2/10, elapsed time: 2.92 sec, test accuracy 93%
Epoch 3/10, elapsed time: 4.20 sec, test accuracy 93%
Epoch 4/10, elapsed time: 5.48 sec, test accuracy 94%
Epoch 5/10, elapsed time: 6.80 sec, test accuracy 95%
Epoch 6/10, elapsed time: 8.10 sec, test accuracy 95%
Epoch 7/10, elapsed time: 9.37 sec, test accuracy 96%
Epoch 8/10, elapsed time: 10.77 sec, test accuracy 95%
Epoch 9/10, elapsed time: 12.55 sec, test accuracy 95%
Epoch 10/10, elapsed time: 14.17 sec, test accuracy 95%
Epoch 1/10, elapsed time: 1.13 sec, test accuracy 93%
Epoch 2/10, elapsed time: 2.38 sec, test accuracy 92%
Epoch 3/10, elapsed time: 3.78 sec, test accuracy 94%
Epoch 4/10, elapsed time: 5.13 sec, test accuracy 95%
Epoch 5/10, elapsed time: 6.43 sec, test accuracy 95%
Epoch 6/10, elapsed time: 7.76 sec, test accuracy 95%
Epoch 7/10, elapsed time: 9.21 sec, test accuracy 95%
Epoch 8/10, elapsed time: 11.12 sec, test accuracy 95%
Epoch 9/10, elapsed tim